# 케글

In [ ]:
# 기본 패키지 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random as rd # generating random numbers
import datetime # manipulating date formats
# 시각화
import matplotlib.pyplot as plt # basic plotting
import seaborn as sns # for prettier plots


# 데이터 확인

In [ ]:
path = os.get
sales = pd.read_csv('')